## Project 2 Content-Based and Collaborative Filtering

## Part 1 Collaborative Filtering

We'll use the Python "surprise" library.  This library offers several packages the support recommender systems, including nearest neighbor-based methods and matrix factoriaztion.  In this section, we'll take advantage of the nearest neighbor algorithms.

In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import gensim
from surprise import Reader
from surprise import Dataset
from surprise.prediction_algorithms.knns import KNNWithZScore, KNNBaseline, KNNBasic
from surprise.prediction_algorithms.baseline_only import BaselineOnly
from surprise.model_selection import GridSearchCV
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import warnings
warnings.filterwarnings('ignore')
import random

random.seed(42)

### Load data

For our data, we'll use a "The Movies Dataset" from kaggle.  This dataset includes a table of 26 million ratings from hundreds of thousands of users.  We subset this data offline because finding a free method of storing this amount of data would be difficult.  Our subset includes all ratings from the top 10,000 users and top 2,000 movies.  The rating scale ranges between .5 and 5 in intervals of .5.

In [17]:
path = 'https://raw.githubusercontent.com/TheFedExpress/DATA612/master/Project2/movies_medium.csv'
movies = (pd.read_csv(path)
          .drop(['Unnamed: 0', 'timestamp'], axis = 1)
          .dropna()
#          .groupby(['userId', 'movieId'])
#          .mean()
#          .reset_index()
)

reader = Reader(rating_scale=(.5, 5))

rec_data = Dataset.load_from_df(movies, reader)
trainset = rec_data.build_full_trainset()

movies.head(5)

,userId,movieId,rating
0,18276.0,4545.0,2.0
1,18276.0,4545.0,2.0
2,18276.0,4205.0,5.0
3,9279.0,8644.0,2.5
4,9279.0,8464.0,4.0


In [18]:
### Data Exploration


In [19]:
avg_user = movies.groupby('userId')['rating'].mean()
layout = {'xaxis' : {'title': 'avg rating'}, 'title' : 'Distribution of Average User Ratings',
         'yaxis': {'title' : 'Users'}
}
data = [go.Histogram(x = avg_user.values)]
py.iplot(go.Figure(data = data, layout = layout))


Most users are somewhat generous with their ratings, with a mode of 4.  Some users are tougher than others, indicating that we might want to standardize ratings by user.

In [20]:
avg_movie = movies.groupby('movieId')['rating'].mean()
layout = {'xaxis' : {'title': 'avg rating'}, 'title' : 'Distribution of Average Movie Ratings',
         'yaxis': {'title' : 'Movies'}
}
data = [go.Histogram(x = avg_movie.values)]
py.iplot(go.Figure(data = data, layout = layout))

The movie ratings are a little less right-skewed.  There is a fair amount of weight between 1 and 3, indicating there are some unpopular movies in our data.  

### Build Models


The Surprise library frames recomendations a bit more like a regression problem than the recommenderlab package.  This is the reason for its sklearn-like api.  The dataset is the input and predicted ratings are the featured output attribute.  **This framework can still be used for recommendations by choosing items with the highest predicted rating.**

We'll try 4 model types, 3 of which use a similarity matrix and take the K most similar users/items, however the expected rating is computed with slightly different formulas.  The last model type will be the baseline model.

- KNNBasic: Raw ratings are weighted by similarities.  Item-based uses the given user's rating of the similar items, while user-based computes the expected rating of an item based on the similar user's rating of that item
- KNNWithZScore: Similar to basic, but starts with the mean of a user/item and adds Z-scores weighted by similarity
- KNNBaseline: Starts with the base line rating and adds (baseline rating - actual rating) of similar items/users weighte by similarity

In testing the KNN models, we'll search over several hyperparameters to compare similarity measures, K, and user-based vs item-based.

In [21]:
param_grid = {'k' : [30,40,50], 
              'sim_options' : {'user_based': [False, True], 
              'name': ['cosine', 'pearson', 'pearson_baseline']}
              
 }

In [22]:
def cv_mod(algo, data):

    cv = GridSearchCV(algo, param_grid, cv = 3, n_jobs = -1)# set cv folds to 3 in the interst of time
    cv.fit(data)

    full_frame = pd.DataFrame.from_dict( cv.cv_results)
    results_df = pd.concat([full_frame[['mean_test_rmse', 'mean_test_mae', 'param_k']], 
               json_normalize(cv.cv_results['param_sim_options'])], axis = 1)
    return cv, results_df


In [23]:
def graph_mod(results_df, algo):

    fig = tools.make_subplots(rows=1, cols=2, subplot_titles = ('Item Based', 'User Based'))
    i = 1
    for filter_type in  results_df.user_based.unique():
        for item in results_df.name.unique():
            df_temp = results_df.loc[(results_df.user_based == filter_type) & (results_df.name == item)]
            trace = go.Scatter(
                x = df_temp.param_k,
                y = df_temp.mean_test_rmse,
                mode = 'lines+markers',
                name = item,
            )
            fig.append_trace(trace,1,i)
            fig['layout']['yaxis1'].update(title = 'RMSE')
            fig['layout']['xaxis' + str(i)].update(title = 'K')
        i+=1
    fig['layout'].update(title=algo)
    return fig

In [24]:
from surprise.model_selection import cross_validate
algo = BaselineOnly()

# Run 5-fold cross-validation and print results
cv1 = cross_validate(algo, rec_data, measures=['RMSE', 'MAE'], cv=5)
pd.DataFrame.from_dict( cv1)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


,test_rmse,test_mae,fit_time,test_time
0,0.750541,0.569211,0.082748,0.051861
1,0.768152,0.583043,0.094717,0.053887
2,0.754012,0.568839,0.110700,0.050894
3,0.744977,0.566414,0.093749,0.050864
4,0.754777,0.575358,0.107711,0.100731


In [25]:
basic_cv, basic_results = cv_mod(KNNBasic, rec_data)
basic_fig = graph_mod(basic_results, 'KNN Basic')
py.iplot(basic_fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [26]:
zscore_cv, zscore_results = cv_mod(KNNWithZScore, rec_data)
z_fig = graph_mod(zscore_results, 'KNN With Z-Score')
py.iplot(z_fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [27]:
baseline_cv, baseline_results = cv_mod(KNNBaseline, rec_data)
base_fig = graph_mod(baseline_results, 'KNN KNNBaseline')
py.iplot(base_fig, layout = layout)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



### Observations

- All 3 KNN-based models outperform the baseline model by a healthy margin
- All 3 KNN-based models agree on the optimal hyperparameters, K equal to 30, item-based, and pearson_baseline as the similarity measure.

The best performing model uses baseline ratings to as both the simiarity measure for the simiarity matrix and computing the expected rating.  This makes sense, as this both encodes more information into the simarity matrix and creates a more complex estimation function.

### Recommend Movies

With the optimal hyperparameters and model-type in hand, we'll build a model using the full training set and show how this model can be used to power a recommendation engine.

We create the "anti_testset", which includes all the item-user combinations not present in the training set.  This can be fed to the model object for predictions.  The predictions the be subsetted for a certain user and ranked for output.  Below, the top 10 recommendations for a given user are shown.

*Note: Due to the metadata table being incomplete, we don't show actual movie names, though it would have been satisfying to do so.*

In [28]:
best_knn = KNNBaseline(k = 30, sim_options = {'name': 'pearson_baseline', 'user_based': False})
best_knn.fit(trainset)

testset = trainset.build_anti_testset()
predictions = (pd.DataFrame(best_knn.test(testset), columns = ['userId', 'movieId', 'act_rating', 'Predicted Rating', 'details'])
    .drop(columns = ['act_rating', 'details'])
)
predictions.loc[predictions.userId == 9279].sort_values('Predicted Rating', ascending = False).head(10)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


,userId,movieId,Predicted Rating
958,9279.0,3347.0,5.000000
1039,9279.0,5525.0,5.000000
1208,9279.0,7616.0,5.000000
1140,9279.0,4789.0,5.000000
720,9279.0,3457.0,5.000000
1240,9279.0,7208.0,5.000000
1061,9279.0,26547.0,5.000000
887,9279.0,4122.0,5.000000
1138,9279.0,5515.0,4.952397
928,9279.0,27741.0,4.920518


## Content-Based Filtering

With unstructured data, content-based filterting can be more difficult than collaborative filtering.  We will only show a simple model architecture for leveraging content-based filtering using LDA.  A more complex hybrid architecture could improve upon the above collaborative filtering approach.

### Load data and initialize libraries

We will be creating a topic model using the overview field from the movie metadata.  Even though the ratings dataset does not include all 42,000 movies from the metadata table, the full dataset will create a better topic model.  Gensim will be our library of choice, though we will also use nltk for stemming and lemmatizing.

Part of the art of topic modeling is choosing an appropriate stopword list.  For optimal results, this list sometimes needs to be hand-curated.  We leave the gender-specific pronouns in the list because they encode some meaning in the movie descriptions.

In [3]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import re
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
import numpy as np

path = 'https://raw.githubusercontent.com/TheFedExpress/DATA612/master/Project2/movies_metadata.csv'
def make_int(df, column):
    df = df.copy()
    df[column] =  df[column].map(lambda x: x.replace('-', ''))
    return df

movies_metadata = (pd.read_csv(path)
                       .loc[:, ['id', 'budget', 'overview', 'runtime']]
                       .pipe(make_int, 'id')
                       .drop_duplicates('id')
)

movies_metadata['id'] = movies_metadata['id'].astype(np.int32)
movies_metadata.head(5)

,id,budget,overview,runtime
0,862,30000000,"Led by Woody, Andy's toys live happily in his ...",81.0
1,8844,65000000,When siblings Judy and Peter discover an encha...,104.0
2,15602,0,A family wedding reignites the ancient feud be...,101.0
3,31357,16000000,"Cheated on, mistreated and stepped on, the wom...",127.0
4,11862,0,Just when George Banks has recovered from his ...,106.0


In [30]:


def lemmatize_stemming(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(desc):
    words = []
    try:
        for item in simple_preprocess(desc, min_len = 3):
             if item not in STOPWORDS or item in ['he', 'she', 'her', 'his']:
                words.append(lemmatize_stemming(item))
        return words
    except(TypeError):
        return np.nan

movies_metadata['movie_words'] = movies_metadata['overview'].map(preprocess)
movie_words = movies_metadata[['id', 'movie_words']].dropna()

### Build LDA Model

We will build our model with 50 topics.  This number must be chosen apriori and usually takes some trial and error to get right.
The most iteresting function parameters here are "no_above", "no_below".  "no_above" filters words from our corpus that only appear 2 or less times.  This smooths over some of the noise.  "no_above" removes words that appear in more than 50% of the documents.  This essentially identifies words that don't provide much information in our corpus, but are not in the stopword list.

In [31]:
dictionary = corpora.Dictionary(movie_words['movie_words'])
dictionary.filter_extremes(no_below=2, no_above=0.5, keep_n=20000)
dictionary.compactify()
corpus = [dictionary.doc2bow(item) for item in movie_words['movie_words']]

In [32]:
lda = models.LdaModel(corpus = corpus, num_topics = 50, id2word = dictionary, passes = 10, alpha = .0005)

### Examine output

We'll look at the top words from the top 10 topics.  Here, we are looking for coherence; are the words related?  What story are they telling.  The first topic, for instance, seems to relate to thrillers and spy movies.

In [33]:
output = lda.print_topics(num_topics = 10)
topics = [lda.get_document_topics(element) for element in corpus]
output

[(28,
  '0.036*"mysteri" + 0.035*"his" + 0.033*"investig" + 0.031*"murder" + 0.022*"secret" + 0.022*"agent" + 0.017*"detect" + 0.013*"kill" + 0.013*"disappear" + 0.012*"suspect"'),
 (35,
  '0.053*"killer" + 0.028*"serial" + 0.022*"isol" + 0.019*"lock" + 0.018*"intim" + 0.016*"safe" + 0.015*"stone" + 0.014*"wall" + 0.013*"devot" + 0.012*"billi"'),
 (21,
  '0.043*"master" + 0.029*"gold" + 0.022*"engin" + 0.020*"shadow" + 0.018*"inspector" + 0.018*"weekend" + 0.016*"simon" + 0.016*"jesus" + 0.016*"upsid" + 0.015*"imprison"'),
 (38,
  '0.085*"school" + 0.067*"new" + 0.061*"high" + 0.046*"student" + 0.031*"york" + 0.030*"girl" + 0.030*"teacher" + 0.024*"colleg" + 0.020*"citi" + 0.015*"friend"'),
 (15,
  '0.101*"his" + 0.019*"get" + 0.014*"money" + 0.012*"want" + 0.012*"friend" + 0.011*"job" + 0.011*"man" + 0.011*"time" + 0.011*"work" + 0.010*"tri"'),
 (49,
  '0.060*"murder" + 0.054*"polic" + 0.037*"crime" + 0.032*"offic" + 0.018*"case" + 0.017*"innoc" + 0.016*"kill" + 0.015*"accus" + 0.014*

These topics appear to be fairly coherent.

### Assign Topic Percentage to Each Movie

This information is a by-product of the topic model.  Each document in the corpus is assigned topic percentages that add up to 100%.  In the LDA model, this represents the probability of a word belonging to a given topic given a particular document.  These are the main features we will be using for our content-based recommender.

In [37]:
all_clus = []
for i in range(len(topics)):
    for j in range(len(topics[i])):
        if topics[i][j][1] >= .1:
            all_clus.append({"id": movie_words.iloc[i, 0], "top_num":topics[i][j][0], "percentage":topics[i][j][1]})

In [38]:
new_df = (pd.DataFrame(all_clus)
              .pivot(index = 'id', columns = 'top_num', values = 'percentage')
)
full_df = movies_metadata.merge(new_df, on = 'id').rename(columns = {'id': 'movieId'})
full_df.head(5)

,movieId,budget,overview,runtime,movie_words,0,1,2,3,4,...,40,41,42,43,44,45,46,47,48,49
0,862,30000000,"Led by Woody, Andy's toys live happily in his ...",81.0,"[lead, woodi, andi, toy, live, happili, his, r...",NaN,NaN,NaN,NaN,NaN,...,NaN,0.116272,NaN,NaN,NaN,0.135838,NaN,NaN,NaN,NaN
1,8844,65000000,When siblings Judy and Peter discover an encha...,104.0,"[sibl, judi, peter, discov, enchant, board, ga...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15602,0,A family wedding reignites the ancient feud be...,101.0,"[famili, wed, reignit, ancient, feud, door, ne...",NaN,NaN,NaN,NaN,NaN,...,0.215810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31357,16000000,"Cheated on, mistreated and stepped on, the wom...",127.0,"[cheat, mistreat, step, women, hold, breath, w...",NaN,0.241188,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11862,0,Just when George Banks has recovered from his ...,106.0,"[georg, bank, recov, his, daughter, wed, recei...",NaN,NaN,NaN,NaN,NaN,...,0.149779,NaN,NaN,0.135961,NaN,NaN,0.279927,NaN,NaN,NaN


### Build Recommendation Model

In this simple example, we'll frame recommendation as a supervised machine learning problem.  The LDA topics, budget, and runtime will be our features and user ratings will be our target.  To further simplify things, we'll assume each user has a separate model based on their past rating history.  XGBoost will be used as our regressor.

A slightly more complex architecture would be to use XGBoost as a feature transformer for the movie content and input that into a mixed model with the user as a random effect.  That would somewhat mitigate the cold-start issue.  That approach is discussed here:

https://engineering.linkedin.com/blog/2019/04/ai-behind-linkedin-recruiter-search-and-recommendation-systems

In [39]:
movies.groupby('userId').size().sort_values(ascending = False).head(5)#Select user for example

userId
533.0    2970
741.0    2233
483.0    2231
543.0    1904
557.0    1728
dtype: int64

In [40]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

final_df = movies.merge(full_df, on = 'movieId')

def impute_median(df, column):
    df = df.copy()
    df[column] = df.loc[df[column] == 0, column].median()
    return df

X = (final_df.pipe(impute_median, 'budget')
         .pipe(impute_median, 'runtime')
         .fillna(0)
         .drop(columns = ['rating', 'userId', 'movieId', 'overview', 'movie_words'])
         .loc[final_df['userId'] == 533]
)
y = final_df.loc[final_df['userId'] == 533, 'rating']

xgb_param_grid = {
    'learning_rate': [.01, .05, .1],
    'max_depth': [3,5,7,9],
    'n_estimators': [750, 1000]
    
}
gb = xgb.XGBRegressor(objective = 'reg:linear', subsample = .6, colsample_bytree = .6, nthread = -1)
regressor = GridSearchCV(gb, xgb_param_grid, cv = 3, scoring = 'neg_mean_squared_error')
regressor.fit(X, y)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=-1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.6),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7, 9], 'n_estimators': [750, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [41]:
full_frame = pd.DataFrame.from_dict( regressor.cv_results_)
pd.concat([full_frame[['mean_test_score']], 
               json_normalize(regressor.cv_results_['params'])], axis = 1)

,mean_test_score,learning_rate,max_depth,n_estimators
0,-0.669251,0.01,3,750
1,-0.681718,0.01,3,1000
2,-0.664323,0.01,5,750
3,-0.667708,0.01,5,1000
4,-0.671259,0.01,7,750
5,-0.673228,0.01,7,1000
6,-0.679478,0.01,9,750
7,-0.681272,0.01,9,1000
8,-0.672301,0.05,3,750
9,-0.672297,0.05,3,1000


## Conclusion

The collaborative filtering approach seemed viable, almost right out of the box.  The RMSE for predicted ratings was less than half a star and making recommendations is as easy as sorting predicted ratings.  Content-based would need a fair amount of additional work to beat that standard, but it does have its advantages.  The content-based seems more scalable as the userbase grows because similarity matricies do not need to be computed.  A hybrid of the two would likely lead to the best performance, as each provides unique information explaining the variance in user ratings.

Sources:
    
- data: https://www.kaggle.com/rounakbanik/the-movies-dataset
- LDA preprocessing: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
- Surprise documentation: https://surprise.readthedocs.io/en/stable/